In [ ]:
!pip install rotman-ncs

⚠️ Restart runtime after package installed

In [60]:
import pandas as pd
import math
import numpy as np

## A4-Q2

You are asked to use the TF-IDF formulaes learnt in the class to convert a statement into TF-IDF vector. The statement is extracted and assigned to variable `doc` in the first code block below. We also tokenize the document into tokens by splitting with whitespace and assign it to variable `tokens` which is a list of tokens. 

We use a copus of 362,330 earning call statements from 2018 to 2021. The total document counts is assigned to `total_doc_in_corpus` variable in the second code block below.

We generate a dictionary of token document counts for the top 1,000 frequent tokens in the corpus, the data is in the "token_doc_counts_a4_q2.csv" file. It is loaded into the variable `token_doc_cnt_dict` variable in the third code block.

Your task is to calculate the normalized TF-IDF score for each token and construct a TF-IDF vector for the given document. Save the TF-IDF vector to "RSM8301-A4-Q2.csv".  

**Note**: You have to remove the tokens that are not in the top 1,000 frequent token list.




In [44]:
# Load call statement and tokenize it.
import ncs
train_call_statements = ncs.load_call_statements('train')
doc = train_call_statements[
    train_call_statements.statement_uid=='9ab2f096-a0fa-4367-ad66-10d8133aafa5'
].clean_text.values[0]
tokens = doc.split(' ')
print(len(tokens))

419


In [3]:
# Total number of documents in the corpus
total_doc_in_corpus = len(train_call_statements)

In [6]:
# Number of documents each token appears in the corpus
token_doc_cnt_dict = pd.read_csv("token_doc_counts_a4_q2.csv",
                                index_col=0).to_dict()["doc_count"]

In [61]:
# Step 1: remove all tokens that are not in the top 1000 list: 
token_filtered = []
for tk in tokens:
    if tk in list(token_doc_cnt_dict.keys()):
        token_filtered.append(tk)

# Step 2: Term Frequency: 
token_count = pd.Series(token_filtered).value_counts()
token_freq = token_count / token_count.sum()

# Step 3: Inverse Document Frequency:
idf = {token: math.log(total_doc_in_corpus / token_doc_cnt_dict[token]) for token in tokens if token in token_doc_cnt_dict}

# Step 4: TF-IDF
tf_idf = {token: token_freq[token] * idf[token] for token in idf}

# Step 5: Normalization:
norm = np.sqrt(sum([x**2 for x in tf_idf.values()]))
tf_idf_normalized = {token: tf_idf[token] / norm for token in tf_idf}

In [62]:
# Write your code here to calculate normalized TF-IDF vector
norm_tfidf = pd.DataFrame(list(tf_idf_normalized.items()), columns=['Token', 'TF-IDF'])
norm_tfidf.to_csv("RSM8301-A4-Q2.csv", header=False, index=False)